In [1]:
import time
import optuna
import mlflow
import numpy as np
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

# Definindo a seed para o random state
rs = 840

/home/daniel/Documents/preditor_fraude/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Lendo os dados
data_fraud = pd.read_csv("../data/processed/data_fraud.csv", index_col=0)

In [3]:
# Dividindo em variáveis explicativas e target
x = data_fraud.drop("fraude", axis = 1)
y = data_fraud["fraude"]

Vamos relembrar pontos importantes que descobrimos na etapa de análise:

- As variáveis **pais** e **categoria_produto** possuem uma alta cardinalidade.
- As variáveis **pais** e **categoria_produto** possuem muitos valores com contagem de categorias iguais.
- Ainda existem variáveis com valores ausentes, tanto categóricas, como numéricas.
- O target está desbalanceado.

Sabendo disso, vamos desenhar como a etapa de experimentação irá se desenrolar:

1. Os dados serão divididos em treino, dev e teste. Iremos treinar o algoritmo
com os dados de treino, fazer a tunagem com os dados de dev, e, por fim, validar
com os dados de teste.
2. Será criado um esqueleto para o pipeline de transformação, consistindo
em um imputer e scaler(quando necessário) para as variáveis numéricas e um imputer 
e um encoder para as categóricas.
    
    2.1. Não usaremos o OneHotEncoder para as colunas com uma alta quantidade de
    categorias únicas, pois isso elevaria a dimensionalidade dos dados.

    2.2. Também não será utilizado o CountEncoder nas colunas com uma alta quantidade
    de categorias únicas, pois algumas categorias apresentam a mesma quantidade de registros.

3. As combinações entre scalers, encoders e modelos serão registradas usando o MLFlow.
4. As métricas avaliadas serão o Recall e a Latência média.

In [14]:
# Define o local para salvar os exoerimentos
mlflow.set_tracking_uri('../mlruns')

# Criando/acessando o experimento
mlflow.set_experiment('Comparando modelos')

<Experiment: artifact_location='/home/daniel/Documents/preditor_fraude/notebooks/../mlruns/809183457967913276', creation_time=1711566682412, experiment_id='809183457967913276', last_update_time=1711566682412, lifecycle_stage='active', name='Comparando modelos', tags={}>

In [15]:
# Divindo os dados em treino, dev e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x,
                                                        y,
                                                        test_size=30,
                                                        stratify=y,
                                                        random_state=rs)

x_dev, x_teste, y_dev, y_teste = train_test_split(x_teste, 
                                                  y_teste,
                                                  stratify=y_teste,
                                                  test_size=0.5,
                                                  random_state=rs)

# Dividindo features numéricas de categóricas
cat_cols_high_dim = ["pais", "categoria_produto"]
cat_cols = [col for col in x_treino.select_dtypes("object").columns if col not in cat_cols_high_dim]
num_cols = x_treino.select_dtypes(["int", "float"]).columns

# Capturando os indices das colunas
cat_cols_index = [x_treino.columns.get_loc(col) for col in cat_cols]
num_cols_index = [x_treino.columns.get_loc(col) for col in num_cols]
cat_cols_high_dim_index = [x_treino.columns.get_loc(col) for col in cat_cols_high_dim]

# Setando o KFold
kf = StratifiedKFold(shuffle=True, random_state=rs)

In [16]:
# Definindo os encoders
# Criando dicionário com os modelos
dict_models_scale_sensitive = {"LR": LogisticRegression(random_state=rs,
                                                        class_weight='balanced'),
                               "SVC": SVC(random_state=rs,
                                          class_weight="balanced")}

dict_models_tree_based = {"LGBM": LGBMClassifier(is_unbalance=True,
                                                 objective= 'multiclass',
                                                 random_state=rs),
                          "XGB": XGBClassifier(random_state=rs,
                                               objective='multi:softprob'),
                          "RF": RandomForestClassifier(class_weight='balanced',
                                                       random_state=rs)}

# Criando dicionário com os encoders
dict_encoders = {"OHE": OneHotEncoder(drop='first'),
                 "TE": ce.TargetEncoder(),
                 "BE": ce.BinaryEncoder(),
                 "ME": ce.MEstimateEncoder(),
                 "CE": ce.CatBoostEncoder(),
                 "GE":ce.GrayEncoder(),
                 "CTE":ce.CountEncoder()}

dict_imputers_num = {"SIAVG": SimpleImputer(strategy='mean'),
                     "SIMEDIAN": SimpleImputer(strategy='median')}

dict_scalers = {"SS": StandardScaler(),
                "RS": RobustScaler()}

# Criando dicionário com os transformers
dict_transformers = {"PT": PowerTransformer()}

In [17]:
# Iniciando os experimentos sem transformers
for tag, model in dict_models_scale_sensitive.items():
    for tag_imputer, imputer in dict_imputers_num.items():
        for tag_encoder, encoder in dict_encoders.items():
            for tag_scaler, scaler in dict_scalers.items():
                
                # Gerando a tag de identificação do modelo
                nome_modelo = f'{tag}_{tag_imputer}_{tag_encoder}_{tag_scaler}'
                
                with mlflow.start_run(run_name=nome_modelo):
                    
                    # Criando os pipeline com os transformers
                    pipe_cat = Pipeline([("imputer_cat", SimpleImputer(strategy='most_frequent')),
                                        ('encoder', encoder)])
                    
                    # Criando os pipeline com os transformers
                    pipe_cat_high_dim = Pipeline([("imputer_cat", SimpleImputer(strategy='most_frequent')),
                                                    ('encoder_hd', ce.CatBoostEncoder())])
                    
                    pipe_num = Pipeline([('imputer_num', imputer),
                                        ('scaler', scaler)])
                    
                    # Criando o transformador
                    transformer = ColumnTransformer([('cat', pipe_cat, cat_cols_index),
                                                    ('num', pipe_num, num_cols_index),
                                                    ('cat_hd', pipe_cat_high_dim, cat_cols_high_dim_index)],
                                                    remainder="passthrough")
                    
                    # Criando o pipeline final
                    pipe = Pipeline([('transformer', transformer),
                                    ('model', model)])
                    
                    # Executando o cross validation
                    cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='recall')
                    
                    # Calculando a média das métricas
                    mean_score = cross_val_scores.mean()           
                    
                    # Salvando a métrica da folder 1
                    mlflow.log_metric('recall_fold_1', cross_val_scores[0])
                    
                    # Salvando a métrica da folder 2
                    mlflow.log_metric('recall_fold_2', cross_val_scores[1])
                    
                    # Salvando a métrica da folder 3
                    mlflow.log_metric('recall_fold_3', cross_val_scores[2])
                    
                    # Salvando a métrica da folder 4
                    mlflow.log_metric('recall_fold_4', cross_val_scores[3])
                    
                    # Salvando a métrica da folder 5
                    mlflow.log_metric('recall_fold_5', cross_val_scores[4])
                    
                    # Salvando as métricas
                    mlflow.log_metric('recall_mean', mean_score)
                    
                    # Treinando o algoritmo
                    pipe.fit(x_treino, y_treino)
                    
                    # Calculando a latência média
                    latency_list = []

                    for _, row in x_treino[:1000].iterrows():

                        # Início da contagem de tempo
                        start_time = time.time()

                        # Extrair os recursos da linha
                        features = row.values.reshape(1, -1)

                        # Fazer a previsão para a linha individual
                        prediction = pipe.predict(features)

                        # Encerra a contagem
                        end_time = time.time()
                        atomic_time = end_time - start_time

                        # Transforma segundo em milissegundo
                        atomic_milissec = atomic_time * 1000

                        # Adiciona o tempo em uma lista
                        latency_list.append(atomic_milissec)

                    # calcula a média 
                    mlflow.log_metric("Latência média", np.mean(latency_list))

/home/daniel/Documents/preditor_fraude/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/daniel/Documents/preditor_fraude/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

KeyboardInterrupt: 